In [1]:
include("/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/calc&codes/sym_tensor/src/SymmetricTensors.jl")  
include("Test.jl")
using Test


Use "(::Type{SymmetricTensor})(...)" instead.


In [2]:
include("/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/calc&codes/sym_tensor/src/SymmetricTensors.jl")
using SymmetricTensors
importall SymmetricTensors


Use "(::Type{SymmetricTensor})(...)" instead.


In [5]:
include("copulagen.jl")
include("naivecum.jl")

naivecumulant (generic function with 2 methods)

In [ ]:
i = 1000000

s = 3

for size in [36, 39]
    data = clcopulagen(i, size);
    @time cumulants(3, data, s);
    @time cumulants(4, data, s);
    @time naivecumulant(data, 4);
end

In [ ]:
data = clcopulagen(120000, 21);

In [10]:
o = 5

@time cumulants(o-1, data, 3);
@time cumulants(o, data, 3);
#@time naivecumulant(data, o);

 31.625345 seconds (3.17 M allocations: 120.542 GB, 17.60% gc time)
261.980526 seconds (53.52 M allocations: 1.008 TB, 12.22% gc time)
39413.502460 seconds (1.67 G allocations: 139.120 TB, 10.17% gc time)


In [ ]:
for o in collect(6:7)
    @time c = cumulants(o, data, 3);
end
#cn = naivecumulant(data, 5);


In [ ]:
 71.781549 seconds (45.45 M allocations: 211.876 GB, 14.92% gc time)
475

#1000000 15 3 
#115.607918 seconds (1.14 M allocations: 343.700 GB, 32.13% gc time)
#3237.854221 seconds (6.58 M allocations: 4.789 TB, 54.23% gc time)
#1000000 30 3 
#1128.944750 seconds (10.66 M allocations: 3.257 TB, 33.08% gc time)
#51451.271712 seconds (105.31 M allocations: 76.620 TB, 54.45% gc time)
#33
#1582.785217 seconds (14.75 M allocations: 4.534 TB, 34.03% gc time)
#78787.640539 seconds (154.18 M allocations: 112.179 TB, 56.60% gc time)
#36
#2146.416321 seconds (19.94 M allocations: 6.152 TB, 34.07% gc time)
#111770.945808 seconds (218.35 M allocations: 158.878 TB, 56.68% gc time)

test = convert(Array, c[4]) - cn
maximum(abs(test))

tweak! (generic function with 1 method)

In [37]:
function tweak!(a::Array{Int})
    a[1] -= 1
    a[2] += 1
end

function h4(a::Array{Int})
    for j in 3:length(a)
        if a[j] < (a[1])
            s = sum(a[1:(j-1)])-1
            return j, s
        end
    end
    0,0
end

function h56!(a::Array{Int})
    j, s = h4(a)
    a[j] += 1
    x = a[j]
    j -= 1
    for i in 2:j
        a[i] = x
    end
    a[1] = s - (j-1)*x;
#    a
end

function mypart(n::Int,sigma::Int, rmin::Int = 1)
    ret = Vector{Int}[]
    a = [n-(sigma-1)*rmin, fill(rmin, (sigma-1))..., -1]
    for i in 1:factorial(n)
        if a[2] > a[1]
            h56!(a)
            if a[end] >= 0
                return ret
            end
        else
            push!(ret, [a...][1:(end-1)]) 
            tweak!(a)
        end
    end
    ret
end

function testpart(n::Int, sigma::Int)
    ret = Vector{Int}[]
    for p in partitions(n, sigma)
    (1 in p)? (): (push!(ret, p))
    end
    ret
end
    

testpart (generic function with 1 method)

In [38]:
testpart(6,3)

1-element Array{Array{Int64,1},1}:
 [2,2,2]

In [42]:


for i in 2:7
    println(mypart(15,i,2)-testpart(15,i))
end

[[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
[[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
[[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]
[[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]
[[0,0,0,0,0,0,0]]


In [22]:
initialize(n::Int) = zeros(Int, n), [1 for i in 1:(n-1)], 1
function increase(a::Array, n::Int)
    a[n] += 1
    a
end

function h4(a::Array{Int}, b::Array{Int}, n::Int)
    for j in (n-1):(-1):1
        if a[j] != b[j]
            return j
        end
    end
    0
end

function record1!(ret, a::Array{Int}, m::Int, n::Int, sigma::Int)
    (m + (m == a[n]) == sigma)? push!(ret, [a...]): ()
end

function record!(ret, a::Array{Int}, m::Int, n::Int, sigma::Int)
    (m + (m == a[n]) == sigma)? push!(ret, transfer(n, [a...])): ()
end

function h25!(ret, a::Array{Int}, b::Array{Int}, m::Int, n::Int, sigma::Int)
    record!(ret, a, m, n, sigma)
    for i in 1:n
        a = increase(a, n)
        record!(ret, a, m, n, sigma)
        if (a[n] == m)
            break
        end
    end
    j = h4(a,b,n)
    (j == 1)? (): (a[j] += 1)
    a, j
end

function h6(a::Vector{Int}, b::Vector{Int}, m::Int, n::Int, j::Int)
    m = b[j] + (a[j] == b[j])
    j += 1
    for i in j:(n-1)
        a[i] = 0
        b[i] = m
    end
    a[n] = 0
    j = j+1
    a,b,m
end

function transfer(n::Int, a::Vector{Int})
    ret = Vector{Int}[]
    seq = collect(1:n)
    for k in 0:maximum(a)
        temp = Int[]
        for i in 1:n
            if a[i] == k
                push!(temp, seq[i])
            end
        end
        push!(ret, temp)
    end
    ret
end

function mysetpart(n::Int, sigma::Int)
    ret = Vector{Vector{Int}}[]
    #ret = Vector{Int}[]
    a, b,m = initialize(n)
    for i in 1:factorial(n)
        a, j = h25!(ret, a,b,m,n, sigma)
        if j == 1
            break
        else
            a, b,m = h6(a,b,m,n,j)
        end
    end
    ret
end

mysetpart (generic function with 2 methods)

In [65]:
n = 6
sigma = 3
@time mysetpart(n, sigma) 
#@time collect(partitions(collect(1:n),sigma))

  0.000146 seconds (1.29 k allocations: 105.719 KB)


90-element Array{Array{Array{Int64,1},1},1}:
 [[1,2,3,4],[5],[6]]
 [[1,2,3,5],[4],[6]]
 [[1,2,3],[4,5],[6]]
 [[1,2,3,6],[4],[5]]
 [[1,2,3],[4,6],[5]]
 [[1,2,3],[4],[5,6]]
 [[1,2,4,5],[3],[6]]
 [[1,2,4],[3,5],[6]]
 [[1,2,4,6],[3],[5]]
 [[1,2,4],[3,6],[5]]
 [[1,2,4],[3],[5,6]]
 [[1,2,5],[3,4],[6]]
 [[1,2],[3,4,5],[6]]
 ⋮                  
 [[1,6],[2,4],[3,5]]
 [[1],[2,4,6],[3,5]]
 [[1],[2,4],[3,5,6]]
 [[1,5,6],[2],[3,4]]
 [[1,5],[2,6],[3,4]]
 [[1,5],[2],[3,4,6]]
 [[1,6],[2,5],[3,4]]
 [[1],[2,5,6],[3,4]]
 [[1],[2,5],[3,4,6]]
 [[1,6],[2],[3,4,5]]
 [[1],[2,6],[3,4,5]]
 [[1],[2],[3,4,5,6]]

In [108]:
initialize(n::Int) = zeros(Int, n), [1 for i in 1:(n-1)], 1

function increase(a::Array, n::Int)
    a[n] += 1
    a
end

function h4(a::Array{Int}, b::Array{Int}, n::Int)
    for j in (n-1):(-1):1
        if a[j] != b[j]
            return j
        end
    end
    0
end

function record1!(ret, a::Array{Int}, m::Int, n::Int, sigma::Int)
    (m + (m == a[n]) == sigma)? push!(ret, [a...]): ()
end

function record!(ret, ret1, a::Array{Int}, m::Int, n::Int, sigma::Int)
    if m + (m == a[n]) == sigma
        c = counts([a...], 0:(sigma-1))
        if !(1 in c)
            push!(ret, transfer(n, [a...]))
            push!(ret1, c)
        end
    end
end

function h25!(ret, ret1, a::Array{Int}, b::Array{Int}, m::Int, n::Int, sigma::Int)
    record!(ret, ret1, a, m, n, sigma)
    for i in 1:n
        a = increase(a, n)
        record!(ret, ret1, a, m, n, sigma)
        if (a[n] == m)
            break
        end
    end
    j = h4(a,b,n)
    (j == 1)? (): (a[j] += 1)
    a, j
end

function h6(a::Vector{Int}, b::Vector{Int}, m::Int, n::Int, j::Int)
    m = b[j] + (a[j] == b[j])
    j += 1
    for i in j:(n-1)
        a[i] = 0
        b[i] = m
    end
    a[n] = 0
    j = j+1
    a,b,m
end

function transfer(n::Int, a::Vector{Int})
    ret = Vector{Int}[]
    seq = collect(1:n)
    for k in 0:maximum(a)
        temp = Int[]
        for i in 1:n
            if a[i] == k
                push!(temp, seq[i])
            end
        end
        push!(ret, temp)
    end
    ret
end

function mysetpart(n::Int, sigma::Int)
    ret = Vector{Vector{Int}}[]
    ret1 = Vector{Int}[]
    #ret = Vector{Int}[]
    a, b,m = initialize(n)
    for i in 1:factorial(n)
        a, j = h25!(ret, ret1, a,b,m,n, sigma)
        if j == 1
            break
        else
            a, b,m = h6(a,b,m,n,j)
        end
    end
    ret, ret1, length(ret1)
end


mysetpart (generic function with 2 methods)

In [301]:
n = 5
sigma = 2

@time p = mysetpart(n, sigma)[1]
pt = mysetpart(n, sigma)[1]
@time p1 = indpart(n, sigma)[1]


  0.000047 seconds (230 allocations: 21.594 KB)
  0.000107 seconds (426 allocations: 21.984 KB)


10-element Array{Array{Array{Int64,1},1},1}:
 [[1,2,3],[4,5]]
 [[1,2,4],[3,5]]
 [[1,2,5],[3,4]]
 [[1,2],[3,4,5]]
 [[1,3,4],[2,5]]
 [[1,3,5],[2,4]]
 [[1,3],[2,4,5]]
 [[1,4,5],[2,3]]
 [[1,4],[2,3,5]]
 [[1,5],[2,3,4]]

In [ ]:
function testpartitions(p::Vector{Vector{Vector{Int}}}, p1::Vector{Vector{Vector{Int}}})
    if length(p) != length(p1)
        return false
    end
    for i in 1:length(p) 
        if !(p1[i] in p) 
            return false
        end
        if !(p[i] in p1) 
            return false
        end
    end
    return true
end

n = 5

for n = 4:14, sigma = 2:floor(Int, n/2)
    println(testpartitions(mysetpart(n, sigma)[1], indpart(n, sigma)[1]))
end

true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true


In [120]:
p = mysetpart(6,3)[1]
indpart(6,3)[1][5:7]

3-element Array{Array{Array{Int64,1},1},1}:
 [[1,4],[2,3],[5,6]]
 [[1,3],[2,5],[4,6]]
 [[1,3],[2,6],[4,5]]

In [ ]:


function testarr(arr::Vector{Vector{Vector{Int}}})
    for i in arr
        for j in i
            for k in j
                (k == 0)? (): (return k)
            end
        end
    end
    0
end

for n in 3:4
    println(testarr(mysetpart(n)- collect(partitions(collect(1:n)))))
end


In [ ]:
indpart(6,3)[1]

In [101]:
n = 4
a = zeros(Int, n)
b = [1 for i in 1:n]
ret = Array{Int}[]
push!(ret, [a...]) 
m = 1
b[n] = m
for i in 1:n
    a[n] += 1
    (a[n]==m)? push!(ret, [a...]): (break) 
end
a[n] = m
j = 0
for i in 1:n
    if a[i] == b[i]
        j = i-1
    end
end
(j == 1)? (return ret): (a[j] += 1)
push!(ret, [a...])
for i in j:(n-1)
    a[i] = 0
    b[i] = m
end
j = j + 1


4

In [68]:
collect(combinations(1:4,2,2))

LoadError: LoadError: MethodError: `combinations` has no method matching combinations(::UnitRange{Int64}, ::Int64, ::Int64)
Closest candidates are:
  combinations(::Any, ::Integer)
while loading In[68], in expression starting on line 1

In [104]:
collect(permutations([0,0,1,1]))

24-element Array{Array{Int64,1},1}:
 [0,0,1,1]
 [0,0,1,1]
 [0,1,0,1]
 [0,1,1,0]
 [0,1,0,1]
 [0,1,1,0]
 [0,0,1,1]
 [0,0,1,1]
 [0,1,0,1]
 [0,1,1,0]
 [0,1,0,1]
 [0,1,1,0]
 [1,0,0,1]
 [1,0,1,0]
 [1,0,0,1]
 [1,0,1,0]
 [1,1,0,0]
 [1,1,0,0]
 [1,0,0,1]
 [1,0,1,0]
 [1,0,0,1]
 [1,0,1,0]
 [1,1,0,0]
 [1,1,0,0]

In [110]:
function indpart(n::Int, sigma::Int)
    p = Vector{Vector{Int}}[]
    r = Vector{Int}[]
    for part in partitions(1:n, sigma)
      s = map(length, part)
      if !(1 in s)
        push!(p, part)
        push!(r, s)
      end
    end
    p, r, length(r)
end


indpart (generic function with 1 method)

In [5]:
indpart(8,2)[3]

119

In [58]:
function su2(n)
    ret = 0
    sigma = 2
    for i in 2:(n- 2*sigma+2)
        ret += div(factorial(n), (factorial(i)*factorial(n-i)))
    end
    div(ret, factorial(sigma))
end
function su3(n)
    ret = 0
    sigma = 3
    for i in 2:(n- 2*sigma+2)
        for j in 2:(n- 2*sigma+4-i)
            ret += div(factorial(n), (factorial(i)*factorial(j)*factorial(n-i-j)))
        end
    end
    div(ret, factorial(sigma))
end
function su4(n)
    ret = 0
    sigma = 4
    for i in 2:(n- 2*sigma+2)
        for j in 2:(n- 2*sigma+4-i)
            for k in 2:(n- 2*sigma+6-i-j)
                ret += div(factorial(n), (factorial(i)*factorial(j)*factorial(k)*factorial(n-i-j-k)))
            end
        end
    end
    div(ret, factorial(sigma))
end

function su5(n)
    ret = 0
    sigma = 5
    for i in 2:(n- 2*sigma+2)
        for j in 2:(n- 2*sigma+4-i)
            for k in 2:(n- 2*sigma+6-i-j)
                for l in 2:(n- 2*sigma+8-i-j-k)
                    ret += div(factorial(n), (factorial(i)*factorial(j)*factorial(k)*factorial(l)*factorial(n-i-j-k-l)))
                end
            end
        end
    end
    div(ret, factorial(sigma))
end

su5 (generic function with 1 method)

In [64]:
function su2(n)
    ret = 0
    sigma = 2
    for i in 2:(n- 2*sigma+2)
        ret += div(factorial(n), (factorial(i)*factorial(n-i)))
    end
    div(ret, factorial(sigma))*(sigma-1)
end
function su3(n)
    ret = 0
    sigma = 3
    for i in 2:(n- 2*sigma+2)
        for j in 2:(n- 2*sigma+4-i)
            ret += div(factorial(n), (factorial(i)*factorial(j)*factorial(n-i-j)))
        end
    end
    div(ret, factorial(sigma))*(sigma - 1)

end
function su4(n)
    ret = 0
    sigma = 4
    for i in 2:(n- 2*sigma+2)
        for j in 2:(n- 2*sigma+4-i)
            for k in 2:(n- 2*sigma+6-i-j)
                ret += div(factorial(n), (factorial(i)*factorial(j)*factorial(k)*factorial(n-i-j-k)))
            end
        end
    end
    div(ret, factorial(sigma))*(sigma-1)
end

function su5(n)
    ret = 0
    sigma = 5
    for i in 2:(n- 2*sigma+2)
        for j in 2:(n- 2*sigma+4-i)
            for k in 2:(n- 2*sigma+6-i-j)
                for l in 2:(n- 2*sigma+8-i-j-k)
                    ret += div(factorial(n), (factorial(i)*factorial(j)*factorial(k)*factorial(l)*factorial(n-i-j-k-l)))
                end
            end
        end
    end
    div(ret, factorial(sigma))*(sigma-1)
end

su5 (generic function with 1 method)

In [65]:
println(su2(4))
println(su2(5))
println(su2(6)+su3(6))
println(su2(7)+su3(7))
println(su2(8)+su3(8)+su4(8))
println(su2(9)+su3(9)+su4(9))
println(su2(10)+su3(10)+su4(10)+su5(10))

3
10
55
266
1414
7862
46281


In [52]:
function sum(n)
    ret = 0
    for sigma in 2: floor(Int, n/2)
        ret += indpart(n,sigma)[3]
    end
    ret
end

sum (generic function with 1 method)

In [54]:
for i in 4:10
    println(sum(i))
end

3
10
40
161
714
3424
17721


In [92]:
if nprocs() == 1
    addprocs()
end
@everywhere begin
    include("/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/calc&codes/sym_tensor/src/Boxtensors.jl")
    using Boxtensors
end
importall Boxtensors

indices (generic function with 1 method)

In [5]:
ret = 1

1